In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path

if __name__ == "__main__":
    __this_file = Path().resolve() / "actor_critic.ipynb"  # jupyter does not have __file__

In [2]:
import torch

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

if __name__ == "__main__":
    print(DEVICE)

cuda:0


In [3]:
from torch import nn

from continuing_education.policy_gradient_methods.reinforce import collect_episode, SamplePolicy, Action, State

# Actor Critic

So in the REINFORCE algorithm, we experimented with batch updates and saw that while it slowed down the code it led to serious improvements. The size of batching needed to introduce stability in learning is called the sample efficiency of an algorithm, and REINFORCE is particularly bad at it. In Actor-Critic we introduce asynchronicity into the environments episode generation and the policy update.

The Actor-Critic method uses asynchronicity by having neural network "servers" which spawn copies of themselves to each interact with copies of the environment. Trajectories are generated by these servers and the gradient updates are sent back to the main server, which averages them and updates the policy. This is a form of parallelism, and it is a very powerful tool in reinforcement learning.

Another concept that Actor Critic introduces is that it hybridizes policy based methods (the actor) and value based methods (the critic). This gives you many of the advantages of both methods.

The huggingface tutorial on A2C wants us to use stable-baselines3, but I call that cheating. We will implement A2C from scratch using PyTorch.

In [ ]:
class Actor(SamplePolicy):
    """This is exactly the same as our SamplePolicy from REINFORCE, but we need to add a few methods."""
    def copy(self) -> "Actor":
        new_actor = Actor(state_size=self.state_size, action_size=self.action_size, hidden_sizes=self.hidden_sizes)
        new_actor.network.load_state_dict(self.network.state_dict())
        return new_actor
    
    def update(self, gradients: list[nn.Module]):
        raise NotImplementedError("Unsure how this works")

In [ ]:
from continuing_education.value_based_methods.dqn.dqn import QLearningModel


class Critic(QLearningModel):

    def copy(self) -> "Critic":
        new_critic = Critic(state_size=self.state_size, action_size=self.action_size, hidden_sizes=self.hidden_sizes)
        new_critic.network.load_state_dict(self.network.state_dict())
        return new_critic
    
    def update(self, gradients: list[nn.Module]):
        raise NotImplementedError("Unsure how this works")

# References

1. Mnih, V., Badia, A. P., Mirza, M., Graves, A., Lillicrap, T. P., Harley, T., … Kavukcuoglu, K. (2016). Asynchronous Methods for Deep Reinforcement Learning. arXiv [Cs.LG]. Retrieved from http://arxiv.org/abs/1602.01783
2. UNIT 6. ACTOR CRITIC METHODS WITH ROBOTICS ENVIRONMENTS. Hugging Face. (n.d.). https://huggingface.co/learn/deep-rl-course/unit6/introduction